# Wikipediaデータベースを用いたQAデータセット

## 概要

**データセットを以下の2種類のデータとして再構築する**  

`df_qa`: Queryとその回答に必要な参考情報のID
- Query: 質問
- ID: 回答に必要な参考情報のID

`df_ref`:
- ID: 回答に必要な参考情報のID
Reference: 参考情報

## imports

In [1]:
import os
import copy
import pandas as pd

import warnings
warnings.simplefilter('ignore')

## 1. 以下に従ってデータをダウンロード

[AI学習データ構築のバオバブ、 LLM（大規模言語モデル）の精度を高めるRAG用データセット作成サービスを開始　〜サンプルデータの無料配布も](https://prtimes.jp/main/html/rd/p/000000004.000112000.html)

`参考`

以下ではエラーが出るため、ファイルを直接ダウンロードする必要あり。
```python
from datasets import load_dataset
dataset = load_dataset("baobab-trees/wikipedia-human-retrieval-ja")
```

## 2. データの読み込み

In [2]:
import pandas as pd
df_answered = pd.read_json('/Users/macbookpro/Downloads/data_answered.jsonl', orient='records', lines=True)
df_n_answered = pd.read_json('/Users/macbookpro/Downloads/data_not_answered.jsonl', orient='records', lines=True)

In [3]:
df_all = pd.concat([df_answered,df_n_answered],axis=0)

In [4]:
df_all.head(1)

,id,question,answer,references
0,1,経済産業省の役割について知りたい。,{'text': '経済産業省は、日本の行政機関のひとつです。経済および産業の発展ならびに鉱...,"[{'search': {'keywords': ['経済産業省']}, 'page': {..."


## 3. `df_qa` と `df_ref` に分割

In [5]:
df_all  = df_all.rename(columns={"id":"ID","question":"Query","answer":"Answer","references":"Reference"})
df_qa = df_all[["ID","Query","Answer"]]
df_ref = df_all[["ID","Reference"]]

## 4. Answerカラム内のデータ整形

In [6]:
def extract_answer(x):
    if pd.notna(x):
        if "text" in x.keys():
            return x["text"]
    else:
        return x
df_qa["Answer"] = df_qa["Answer"].map(extract_answer)

/var/folders/7f/5z41mf8x4ksbcxbf7vsr0fs40000gn/T/ipykernel_15171/990881788.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qa["Answer"] = df_qa["Answer"].map(extract_answer)


In [7]:
df_qa.head(2)

,ID,Query,Answer
0,1,経済産業省の役割について知りたい。,経済産業省は、日本の行政機関のひとつです。経済および産業の発展ならびに鉱物資源およびエネルギ...
1,2,確定申告のやり方を教えて欲しい。,確定申告書は、申告時点での住所地を管轄する税務署へ郵送するか、直接持参するか、e-Taxで申...


In [8]:
df_ref.head(2)

,ID,Reference
0,1,"[{'search': {'keywords': ['経済産業省']}, 'page': {..."
1,2,"[{'search': {'keywords': ['確定申告']}, 'page': {'..."


## 5. Referenceの(id,ref)のペアの作成

In [12]:
# referenceの(id,ref)のペアの作成
pairs=[]
for i in range(len(df_ref)):
    data = df_ref.iloc[i]
    id = data["ID"]
    refs = data["Reference"]
    for i,r in enumerate(refs):
        pairs.append((str(id),r))
        # pairs.append((str(id).zfill(3)+"-"+str(i).zfill(2),r))

## 6. 情報抽出

In [13]:
# textが入ってるものだけを取り出し
contents=[]
for p in pairs:
    
    search=""
    if "search" in p[1].keys():
        search=p[1]["search"]["keywords"][0]

    title=""
    text=""
    if "page" in p[1].keys():
        if "title" in p[1]["page"].keys():
            title = p[1]["page"]["title"]
        if "quote" in p[1]["page"].keys():
            if "text" in p[1]["page"]["quote"].keys():
                text = p[1]["page"]["quote"]["text"]
                contents.append((p[0],search,title,text))
            else:
                continue

## 7. `df_ref` の作成

In [14]:
# df_refの作成
df_ref = pd.DataFrame(contents,columns=["ID","SearchKeyword","PageTitle","Reference"])
df_ref = df_ref[["ID","Reference","SearchKeyword","PageTitle"]]

In [15]:
df_ref.head(5)

,ID,Reference,SearchKeyword,PageTitle
0,1,"経済産業省（けいざいさんぎょうしょう、英: Ministry of Economy, Tra...",経済産業省,経済産業省
1,1,経済産業省設置法第3条の定める任務である「民間の経済活力の向上及び対外経済関係の円滑な発展を...,経済産業省,経済産業省
2,2,作成した確定申告書は、申告時点での住所地を管轄する税務署へ郵送するか、直接持参するか、e-T...,確定申告,確定申告
3,3,もう1つの論点である経済性では、原材料の枯渇に伴う価格高騰を理由としている。EVは莫大な鉱物...,電気自動車,電気自動車
4,4,李秉喆が早稲田大学中退後、馬山にて友人2人と1万円ずつ出資し設立した協同精米所の事業が失敗、...,サムスン,サムスングループ


In [16]:
df_qa.head(2)

,ID,Query,Answer
0,1,経済産業省の役割について知りたい。,経済産業省は、日本の行政機関のひとつです。経済および産業の発展ならびに鉱物資源およびエネルギ...
1,2,確定申告のやり方を教えて欲しい。,確定申告書は、申告時点での住所地を管轄する税務署へ郵送するか、直接持参するか、e-Taxで申...


In [17]:
df_ref.shape,df_ref["ID"].nunique()

((1651, 4), 818)

## 8. 保存

In [19]:
import os
dataset_dir = "./dataset/Wikipedia"
os.makedirs(dataset_dir,exist_ok=True)
df_ref.to_csv(dataset_dir+"/Reference.csv")
df_qa.to_csv(dataset_dir+"/QA.csv")